In [114]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz --user
nlp = en_core_web_sm.load()
import pandas as pd
import json
!pip install bs4 --user
from bs4 import BeautifulSoup
import requests
import re
import os
import mmap

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Load data

In [91]:
with open('train_data.json', 'r') as f:
    dict_json_weighted = json.load(f)
object_1 = list(dict_json_weighted.keys())[4]
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(str(i)+"-"+dict_json_weighted[object_1]['log'][i]['text'])

0-I am looking for information about the City Centre North B and B hotel.  Can you help me with that?
1-Yes! City Centre North B and B is an inexpensively-priced guesthouse in the north of town at 328A Histon Road. Would you like to book a room?
2-Yes can I book it for 6 people starting on Saturday for 4 nights please?
3-They weren't able to accommodate those parameters. Would you like to try a shorter stay?
4-Sure.  Let's try 2 nights.  Thanks. I'll need a reference number, too, please.
5-The booking for City Centre North B&B has been made, your reference number is 0FAFY5PP.
6-Great! I also need a train from bishops stortford to cambridge, please.
7-I can find one for you. Can you tell me what day you would like to travel, please? 
8-The train should arrive by 14:45 and should be on the same day as the hotel booking.
9-Great, I recommend train TR5108.  It departs at 13:29 and will get you there by 14:07.  Would you like me to book it?
10-Please book that train for me now.
11-Certainly

## Apply nlp for each sentence 

In [139]:
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(str(i)+"-"+dict_json_weighted[object_1]['log'][i]['text'])
    sentence =nlp (dict_json_weighted[object_1]['log'][i]['text'])
    ### create a file for each sentence
    result= open("Data/"+object_1+str(i)+".txt","w+")
    ### print the label
    #print("Verbs:", [token.lemma_ for token in sentence if token.pos_ == "VERB"])
    for X in (sentence.ents) :
        print([(X.text, X.label_) ])
        r = open("Data/"+object_1+str(i)+".txt", "r")     
        if str(X.label_) not in r.read():
            print(X.label_+"**else")
            result.write("~["+X.label_+"]\n")
            result.write("   "+X.text+"\n")
        elif "~["+X.label_+"]" in r.read():
            print(X.label_+"**if")
            result.write("   "+X.text+"\n")
                

        
    #labels = [x.label_ for x in sentence.ents]
    #print(len(sentence.ents))
    ### print the labels in each sentence
    #print(Counter(labels))
    ### highlight named entities and their labels in a text
    displacy.render(sentence, jupyter=True, style='ent')
    ### display the sentence and its dependencies 
    #displacy.render(sentence, style='dep', jupyter = True, options = {'distance': 120})
    
    ###apply part-of-speech tagging and lemmatize the sentence.
    for x in [y for y in sentence if y.pos_ != 'PUNCT']:
        print ((x.orth_,x.pos_, x.lemma_))
    print ("******************************************************")
    

0-I am looking for information about the City Centre North B and B hotel.  Can you help me with that?
[('the City Centre North B', 'GPE')]
GPE**else


('I', 'PRON', '-PRON-')
('am', 'AUX', 'be')
('looking', 'VERB', 'look')
('for', 'ADP', 'for')
('information', 'NOUN', 'information')
('about', 'ADP', 'about')
('the', 'DET', 'the')
('City', 'PROPN', 'City')
('Centre', 'PROPN', 'Centre')
('North', 'PROPN', 'North')
('B', 'PROPN', 'B')
('and', 'CCONJ', 'and')
('B', 'PROPN', 'B')
('hotel', 'NOUN', 'hotel')
(' ', 'SPACE', ' ')
('Can', 'AUX', 'can')
('you', 'PRON', '-PRON-')
('help', 'VERB', 'help')
('me', 'PRON', '-PRON-')
('with', 'ADP', 'with')
('that', 'DET', 'that')
******************************************************
1-Yes! City Centre North B and B is an inexpensively-priced guesthouse in the north of town at 328A Histon Road. Would you like to book a room?
[('Histon Road', 'FAC')]
FAC**else


('Yes', 'INTJ', 'yes')
('City', 'PROPN', 'City')
('Centre', 'PROPN', 'Centre')
('North', 'PROPN', 'North')
('B', 'PROPN', 'B')
('and', 'CCONJ', 'and')
('B', 'PROPN', 'B')
('is', 'AUX', 'be')
('an', 'DET', 'an')
('inexpensively', 'ADV', 'inexpensively')
('priced', 'VERB', 'price')
('guesthouse', 'NOUN', 'guesthouse')
('in', 'ADP', 'in')
('the', 'DET', 'the')
('north', 'NOUN', 'north')
('of', 'ADP', 'of')
('town', 'NOUN', 'town')
('at', 'ADP', 'at')
('328A', 'NUM', '328a')
('Histon', 'PROPN', 'Histon')
('Road', 'PROPN', 'Road')
('Would', 'AUX', 'would')
('you', 'PRON', '-PRON-')
('like', 'VERB', 'like')
('to', 'PART', 'to')
('book', 'VERB', 'book')
('a', 'DET', 'a')
('room', 'NOUN', 'room')
******************************************************
2-Yes can I book it for 6 people starting on Saturday for 4 nights please?
[('6', 'CARDINAL')]
CARDINAL**else
[('Saturday', 'DATE')]
DATE**else
[('4 nights', 'TIME')]
TIME**else


('Yes', 'INTJ', 'yes')
('can', 'AUX', 'can')
('I', 'PRON', '-PRON-')
('book', 'VERB', 'book')
('it', 'PRON', '-PRON-')
('for', 'ADP', 'for')
('6', 'NUM', '6')
('people', 'NOUN', 'people')
('starting', 'VERB', 'start')
('on', 'ADP', 'on')
('Saturday', 'PROPN', 'Saturday')
('for', 'ADP', 'for')
('4', 'NUM', '4')
('nights', 'NOUN', 'night')
('please', 'INTJ', 'please')
******************************************************
3-They weren't able to accommodate those parameters. Would you like to try a shorter stay?


/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


...

('Certainly', 'ADV', 'certainly')
('How', 'ADV', 'how')
('many', 'ADJ', 'many')
('tickets', 'NOUN', 'ticket')
('would', 'AUX', 'would')
('you', 'PRON', '-PRON-')
('like', 'VERB', 'like')
******************************************************
12-I need 6 tickets for the same day.
[('6', 'CARDINAL')]
CARDINAL**else
[('the same day', 'DATE')]
DATE**else


('I', 'PRON', '-PRON-')
('need', 'VERB', 'need')
('6', 'NUM', '6')
('tickets', 'NOUN', 'ticket')
('for', 'ADP', 'for')
('the', 'DET', 'the')
('same', 'ADJ', 'same')
('day', 'NOUN', 'day')
******************************************************
13-Booking was successful, the total fee is 48.48 GBP payable at the station .
Reference number is : GSTIN0L6.
[('48.48', 'CARDINAL')]
CARDINAL**else


('Booking', 'NOUN', 'booking')
('was', 'AUX', 'be')
('successful', 'ADJ', 'successful')
('the', 'DET', 'the')
('total', 'ADJ', 'total')
('fee', 'NOUN', 'fee')
('is', 'AUX', 'be')
('48.48', 'NUM', '48.48')
('GBP', 'PROPN', 'GBP')
('payable', 'ADJ', 'payable')
('at', 'ADP', 'at')
('the', 'DET', 'the')
('station', 'NOUN', 'station')
('\n', 'SPACE', '\n')
('Reference', 'PROPN', 'Reference')
('number', 'NOUN', 'number')
('is', 'AUX', 'be')
('GSTIN0L6', 'PROPN', 'GSTIN0L6')
******************************************************
14-That's all I need today, thank you. 
[('today', 'DATE')]
DATE**else


('That', 'DET', 'that')
("'s", 'AUX', 'be')
('all', 'DET', 'all')
('I', 'PRON', '-PRON-')
('need', 'VERB', 'need')
('today', 'NOUN', 'today')
('thank', 'VERB', 'thank')
('you', 'PRON', '-PRON-')
******************************************************
15-You're welcome! Lovely day to you.
[('Lovely day', 'DATE')]
DATE**else


('You', 'PRON', '-PRON-')
("'re", 'AUX', 'be')
('welcome', 'ADJ', 'welcome')
('Lovely', 'ADJ', 'lovely')
('day', 'NOUN', 'day')
('to', 'ADP', 'to')
('you', 'PRON', '-PRON-')
******************************************************
